In [7]:
pip install psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 11.8 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install spacy

  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl.metadata (2.8 kB)
  Using cached spacy_loggers-1.0.5-py3-none-any.whl.metadata (23 kB)
  Using cached wasabi-1.1.3-py3-none-any.whl.metadata (28 kB)
  Using cached catalogue-2.0.10-py3-none-any.whl.metadata (14 kB)
  Using cached weasel-0.4.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached langcodes-3.5.0-py3-none-any.whl.metadata (29 kB)
  Using cached language_data-1.3.0-py3-none-any.whl.metadata (4.3 kB)
  Using cached confection-0.1.5-py3-none-any.whl.metadata (19 kB)
  Using cached cloudpathlib-0.21.0-py3-none-any.whl.metadata (14 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 8.6 MB/s eta 0:00:00a 0:00:01
Using cached catalogue-2.0.10-py3-none-any.whl (17 kB)
Using cached langcodes-3.5.0-py3-none-any.whl (182 kB)
Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl (29 kB)
Using cached spacy_loggers-1.0.5-py3-none-any.whl (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 634.7/634.7 kB 18.7 MB/s eta 0:00:00


In [17]:
import psycopg2
# Database connection config
DB_CONFIG = {
    "host": "localhost",
    "port": 5432,
    "dbname": "postgres",
    "user": "postgres",
    "password": "postgres"
}

# Connect to PostgreSQL
conn = psycopg2.connect(**DB_CONFIG)
cursor = conn.cursor()

In [18]:

# Create new table if not exists
cursor.execute("""
    CREATE TABLE IF NOT EXISTS himachal_headlines (
        id SERIAL PRIMARY KEY,
        headline TEXT,
        matched_keywords TEXT[],
        original_article_id INTEGER,
        added_at TIMESTAMP DEFAULT NOW()
    );
""")
conn.commit()

In [8]:
# Fetch articles
cursor.execute("SELECT id, headline FROM nyt_articles WHERE headline IS NOT NULL;")
articles = cursor.fetchall()

In [11]:
print(len(articles))

7332


In [4]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 7.7 MB/s eta 0:00:00a 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [5]:
import spacy

# Load spaCy NLP model
nlp = spacy.load("en_core_web_sm")

In [9]:



# Define tourism-related keywords
TOURISM_KEYWORDS = [
    "Himachal", "Himachal Pradesh", "Shimla", "Manali", "Spiti", "Dharamshala", 
    "Kullu", "tourism", "hill station", "travel", "trek", "snowfall", "adventure", 
    "eco-tourism", "heritage", "mountains", "nature"
]

# Fetch articles
cursor.execute("SELECT id, headline FROM nyt_articles WHERE headline IS NOT NULL;")
articles = cursor.fetchall()

# Process and filter
for article_id, headline in articles:
    doc = nlp(headline)
    headline_lower = headline.lower()
    
    matched_keywords = [kw for kw in TOURISM_KEYWORDS if kw.lower() in headline_lower]
    
    if matched_keywords:
        cursor.execute("""
            INSERT INTO himachal_headlines (headline, matched_keywords, original_article_id)
            VALUES (%s, %s, %s)
            ON CONFLICT DO NOTHING;
        """, (headline, matched_keywords, article_id))

conn.commit()
cursor.close()
conn.close()
print("NER and insertion complete.")

NER and insertion complete.


In [19]:
cursor.execute("SELECT * FROM himachal_headlines;")
articles = cursor.fetchall()
articles

[(1,
  'An Imaginative Treat for ‘Adventure Time’ Fans',
  ['adventure'],
  334,
  datetime.datetime(2025, 4, 12, 5, 27, 6, 620909)),
 (2,
  '‘Let’s Make a Dance.’ At Nature Theater, the Body Rules.',
  ['nature'],
  404,
  datetime.datetime(2025, 4, 12, 5, 27, 6, 620909)),
 (3,
  'Quote of the Day: Jolted by War, Ukrainians Are Reclaiming Their Literary Heritage',
  ['heritage'],
  551,
  datetime.datetime(2025, 4, 12, 5, 27, 6, 620909)),
 (4,
  'The Stewards of Czech Musical Heritage Return to Carnegie Hall',
  ['heritage'],
  822,
  datetime.datetime(2025, 4, 12, 5, 27, 6, 620909)),
 (5,
  'South Korea Bars President From Traveling Abroad',
  ['travel'],
  1069,
  datetime.datetime(2025, 4, 12, 5, 27, 6, 620909)),
 (6,
  'Chasing the $100 Lift Ticket: 6 Mountains Where You Can Ski for Less',
  ['mountains'],
  1228,
  datetime.datetime(2025, 4, 12, 5, 27, 6, 620909)),
 (7,
  '‘Nature of the Crime’ Review: Yearning for a Second Chance',
  ['nature'],
  1295,
  datetime.datetime(2025,